In [ ]:

"""
This program creates the html pages with tick's increment in daily, weekly, or monthly basis
"""

#libraries
import pandas as pd
from datetime import datetime
import sys
import os
sys.path.append('/home/thakur/test')
import html_template as html
import numpy as np


In [ ]:
# nas=pd.read_csv('/home/thakur/stock_information/nasdaq_all.csv')
# nas[nas.Symbol=='NIO']
# us_com=nas[nas.Country=='United States'].Symbol.to_list()
# us_com

In [ ]:
nas=pd.read_csv('/home/thakur/stock_information/nasdaq_all.csv')
us_com=nas[nas.Country=='United States'].Symbol.to_list()
us_com[:10]

In [ ]:
#today
today=datetime.now().strftime('%m%d%y')
#today='100722'
def get_status_df(file,cat,days=3):
    """
    Checks whether the df is increasing for days, weeks and months and returns the list.
    """
    #read the file
    df_day=pd.read_csv(file)
    df_day['Date']=df_day['Date'].apply(lambda x:x.split()[0])
    df_day["Date"] = pd.to_datetime(df_day["Date"])
    #df_day["Date"]=df_day["Date"].apply(lambda x:x.strftime("%Y-%m-%d"))
    df_day.set_index("Date", inplace=True)
    #df_day.index=pd.to_datetime(df_day.index,unit='s')
    #tick
    tick=file.split('.')[0].split('/')[-1]

    #get the dataframe
    df_daily=df_day['Adj Close'].tail(days)
    df_monthly=df_day['Adj Close'].resample(rule='BM').last().tail(days)
    df_weekly=df_day['Adj Close'].resample(rule='W').last().tail(days+1)
    #print(50*'==')
    #print("Daily  :\n",df_daily)
    #print("Weekly :\n",df_weekly)
    #print("Monthly:\n",df_monthly)

    #checking the status
    month_increasing=df_monthly.is_monotonic_increasing
    month_decreasing=df_monthly.is_monotonic_decreasing
    week_increasing=df_weekly.is_monotonic_increasing
    week_decreasing=df_weekly.is_monotonic_decreasing
    day_increasing=df_daily.is_monotonic_increasing
    day_decreasing=df_daily.is_monotonic_decreasing

    chk_day="up" if day_increasing else "down" if day_decreasing else "none"
    chk_week="up" if week_increasing else "down" if week_decreasing else "none"

    chk_month="up" if month_increasing else "down" if month_decreasing else "none"
    #print([tick,chk_day,chk_week,chk_month])
    return [tick,chk_day,chk_week,chk_month,cat]


In [ ]:
root='/home/thakur/test/{}/'
dirs=['mega','medium','large','small','micro','nano']
root,dirs

In [ ]:
all_list=[]
for i in dirs:
    files=os.listdir(root.format(i))
    for f in files:
        # if f.split('.')[0] not in nas:
        #     print(f"{f}: Non American")
        #     continue
        all_list.append(get_status_df(root.format(i)+f,i))
        

In [ ]:
#all_list

In [ ]:
col_names=['TICKER','DAYS_UP','WEEK_UP','MONTH_UP','CATEGORY']

df=pd.DataFrame(all_list,columns=col_names)

In [ ]:
df

In [ ]:
df=df[df.TICKER.isin(us_com)].reset_index(drop=True)
df

# DAYS, WEEK, MONTH UP

In [ ]:
cond=(df['DAYS_UP']=='up')&(df['WEEK_UP']=='up')&(df['MONTH_UP']=='up')
df_up=df[cond]
df_up=df_up\
.reset_index(drop=True)
df_up

In [ ]:
grouped=df_up.groupby(['CATEGORY'])


In [ ]:
def get_last_price(ticker,cat):
    '''
    returns the last price of the ticker
    '''
    f=root.format(cat)+ticker+'.csv'
    last_val=pd.read_csv(f).tail(1)['Adj Close'].values[0]
    return last_val
    

In [ ]:
def get_yahoo_link(ticker):
    '''returns yahoo link to the given ticker'''
    return f'https://finance.yahoo.com/quote/{ticker}?p={ticker}&.tsrc=fin-srch'

def get_google_link(ticker):
    '''returns yahoo link to the given ticker'''
    return f'https://www.google.com/finance/quote/{ticker}'
    #return f'https://finance.yahoo.com/quote/{ticker}?p={ticker}&.tsrc=fin-srch'

In [ ]:
# nasdaq=nas[['Symbol','Country']]
# nasdaq
# #nasdaq.Country.value_counts()
# #.values_count()

In [ ]:
for dir in dirs:
    temp=grouped.get_group(dir).reset_index(drop=True)
    #temp['LAST']=temp['TICKER'].apply(lambda x:get_last_price(x,dir))
    temp.insert(1,'LAST',temp['TICKER'].apply(lambda x:get_last_price(x,dir)))
    temp['Y-LINK']=temp['TICKER'].apply(lambda x:get_yahoo_link(x))
    temp['G-LINK']=temp['TICKER'].apply(lambda x:get_google_link(x))
    #temp['COUNTRY']=nasdaq[nasdaq.Symbol==
    temp=temp.sort_values(by='LAST').reset_index(drop=True)
    temp.index+=1
    print(f"{temp.to_string()}\n")